In [5]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
import torch
import os.path
from spacy.lang.en import English
import torch.nn.functional as F
from src.models.nliclassifier import NLIClassifier
from src.dataset.dataloaders import get_embeddings_for_data
from src.dataset.snli_dataset import str_to_idxs, label_map

In [7]:
# Make sure to enter the parent dir of the embedding vocab+vector file used for training
emb_vocab, emb_vecs = get_embeddings_for_data(
    dataset_path=os.path.join("..", "data", "processed")
)

In [8]:
# Load the model along with the embeddings
model = (
    NLIClassifier.load_from_checkpoint(
        os.path.join(
            "..",
            "checkpoint",
            "real_blstmpme_train",
            "epoch=14-step=32190-val_acc=0.86.ckpt",
        ),
        strict=False,
        embedding_mat=emb_vecs,
    )
    .cpu()
    .eval()
)

/Users/patrik/miniforge3/envs/atcs/lib/python3.10/site-packages/lightning/pytorch/core/saving.py:184: Found keys that are in the model state dict but not in the checkpoint: ['embedding.weight']


In [9]:
tokenizer = English().tokenizer

In [10]:
def predict_and_print(sent_1, sent_2):
    raw_inputs = str_to_idxs(sent_1, sent_2, tokenizer, emb_vocab)
    out = model(*raw_inputs)
    probs = F.softmax(out, dim=-1)
    label = torch.argmax(probs, dim=1).unsqueeze(0).detach().item()
    print(
        f"""
    Premise: "{sent_1}"
    Hypothesis: "{sent_2}"
    Model judgement: "{label_map[label]}"
    """
    )

In [11]:
predict_and_print("Two men sitting in the sun", "Nobody is sitting in the shade")


    Premise: "Two men sitting in the sun"
    Hypothesis: "Nobody is sitting in the shade"
    Model judgement: "contradiction"
    


In [12]:
predict_and_print("A man is walking a dog", "No cat is outside")


    Premise: "A man is walking a dog"
    Hypothesis: "No cat is outside"
    Model judgement: "contradiction"
    
